# Задание 7

## 1. PropertyCreator (0.2 балла)

Напишите мета класс для создания свойств (property) класса из функций начинающихся с "set\_", "get\_" или "del_". Пример использования:
<code>
class TestPropertyCreator(metaclass=PropertyCreator):
    def \__init\__(self, lo):
        self.__x = None
        self.lo = lo

    def get_x(self):
        return self.__x

    def set_x(self, value):
        if value < self.lo:
            raise ValueError("Value must in condition: {} <= value".format(self.lo))
        self.__x = value
    
    def del_x(self):
        self.__x = "No more"

    pass


obj = TestPropertyCreator(5)
obj.x = 4
print(obj.x)
del (obj.x)
</code>

* Мета класс должен поддерживать наследование, в смысле создавать свойства у потомков.
* Должен поддерживать частичное описание свойств, т. е. например, описание одного метода get_val (без set_val и del_val).
* Поддерживать множественное использование одного свойства с одним именем в разных классах:

<code>
class A(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class A"

class B(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class B"

class C(metaclass=PropertyCreator):
    def set_x(self, value):
        self.value = "x in class C"
    def get_x(self):
        return self.value
</code>
* Должен уметь обрабатывать имен с несколькими подчеркиваниями "get_raw_text".

#### Решение

In [1]:
class PropertyCreator(type):
    def __new__(mcs, name, bases, attrs):
        properties = {}

        storage = type('storage', (), {'__getattr__': lambda self, attr: None})

        for key, value in attrs.items():
            if len(key) > 4 and key.startswith(('get_', 'set_', 'del_')):
                current_name = key[4:]
                if current_name not in properties:
                    properties[current_name] = storage()
                if key.startswith('get_'):
                    properties[current_name].getter = value
                elif key.startswith('set_'):
                    properties[current_name].setter = value
                elif key.startswith('del_'):
                    properties[current_name].deleter = value

        new_attrs = {key: property(value.getter, value.setter, value.deleter)
                     for key, value in properties.items()}
        return type.__new__(mcs, name, bases, dict(**attrs, **new_attrs))

#### Протестируйте свое решение

In [2]:
def test_simple():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self, lo):
            self.__x = None
            self.lo = lo

        def get_x(self):
            return self.__x

        def set_x(self, value):
            if value < self.lo:
                raise ValueError(
                    "Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"

    obj = TestPropertyCreator(42)
    assert obj.x is None

    obj.x = 100
    assert obj.x == 100

    try:
        obj.x = 5
        assert False
    except ValueError:
        pass

    del obj.x
    assert obj.x == 'No more'

    print('test_simple passed!')


def test_with_inheritance():
    class TestPropertyCreator(metaclass=PropertyCreator):
        pass

    class TestPropertyCreatorInheritance(TestPropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_x(self, value):
            self._secret_list.append("set")

    obj = TestPropertyCreatorInheritance()
    assert obj.x == 0
    assert obj._secret_list == ['get']

    obj.x = 5
    assert obj._secret_list == ['get', 'set']

    assert obj.x == 0
    assert obj._secret_list == ['get', 'set', 'get']

    print('test_with_inheritance passed!')


def test_partially_defined():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_y(self, value):
            self._secret_list.append("set")
            self._y = value

    obj = TestPropertyCreator()
    assert obj.x == 0

    try:
        obj.x = 5
        assert False
    except AttributeError:
        pass
    assert obj._secret_list == ['get']

    try:
        var = obj.y
        assert False
    except AttributeError:
        pass

    obj.y = 45
    assert obj._secret_list == ['get', 'set']
    assert obj._y == 45

    print('test_partially_defined passed!')


def test_sanity():
    class TestPropertyCreator(metaclass=PropertyCreator):
        _text = 0

        def get_raw_text(self):
            return self._boo

        def get_text(self):
            return self._text % 2

        def set_text(self, value):
            try:
                self._text = int(value)
            except ValueError:
                raise TypeError("unproper value for text: {}".format(value))

    obj = TestPropertyCreator()
    obj._boo = 'BOO!!!'
    assert obj.raw_text == 'BOO!!!'

    try:
        obj.text = 'five'
        assert False
    except TypeError:
        pass

    obj.text = '5'
    assert obj.text == 1
    assert obj._text == 5

    print('test_sanity passed!')


def test_multiple_usages():
    class TestPropertyCreatorA(metaclass=PropertyCreator):
        def get_x(self):
            return 0

    class TestPropertyCreatorB(metaclass=PropertyCreator):
        def get_x(self):
            return 1

    class TestPropertyCreatorC(metaclass=PropertyCreator):
        def set_x(self, value):
            self.value = value + 1

        def get_x(self):
            return self.value

    a = TestPropertyCreatorA()
    b = TestPropertyCreatorB()
    c = TestPropertyCreatorC()

    try:
        a.x = 5
        assert False
    except AttributeError:
        pass

    try:
        b.x = 7
        assert False
    except AttributeError:
        pass

    c.x = 9
    assert c.x == 10
    assert b.x == 1
    assert a.x == 0

    print('test_multiple_usages passed!')


test_simple()
test_with_inheritance()
test_partially_defined()
test_sanity()
test_multiple_usages()

test_simple passed!
test_with_inheritance passed!
test_partially_defined passed!
test_sanity passed!
test_multiple_usages passed!


## 2. InstanceCountExeptioner (0.2 балла)
Напишите метакласс InstanceCountExeptioner, который будет следить за количеством экземпляров класса, использующих его. Количество задается через поле класса \_\_max_instane\_count\_\_. Т. е. число экземпляров каждого класса регулируется отдельно. Если в классе не указано поле \_\_max_instane\_count\_\_, то используйте заранее заданное число в метаклассе (любое). Пример:

<code>
class TestInstanceCountExeptionerA(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 2
    def \__init\__(self, a):
        self.a = a


class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 1
    def \__init\__(self, a):
        self.a = a

a_one = TestInstanceCountExeptionerA('one')
a_two = TestInstanceCountExeptionerA('two')
b_one = TestInstanceCountExeptionerB('one')
\# пока всё шло хорошо

\# а вот
a_three = TestInstanceCountExeptionerA('three')
\# выкенет исключение InstanceCountExeption (ваше собственное исключение)
</code>

#### Решение

In [3]:
class InstanceCountException(Exception):
    def __init__(self, msg=''):
        self.msg = msg

    def __str__(self):
        return str(self.msg)

    def __repr__(self):
        return f'InstanceCountException({repr(self.msg)})'

In [4]:
_MAX_INSTANCE_COUNT = '__max_instance_count__'
_INSTANCE_COUNT = '__instance_count__'


class InstanceCountExceptioner(type):
    _default__max_instance_count__ = 1

    def __new__(mcs, name, bases, attrs):
        new_attrs = {_INSTANCE_COUNT: 0}
        if _MAX_INSTANCE_COUNT not in attrs:
            new_attrs[_MAX_INSTANCE_COUNT] = mcs._default__max_instance_count__

        return type.__new__(mcs, name, bases, dict(**attrs, **new_attrs))

    def __call__(cls, *args, **kwargs):
        cnt = getattr(cls, _INSTANCE_COUNT)
        if cnt >= getattr(cls, _MAX_INSTANCE_COUNT):
            raise InstanceCountException(
                'Too many instances of {}'.format(cls.__name__))
        setattr(cls, _INSTANCE_COUNT, cnt + 1)
        return type.__call__(cls, *args, **kwargs)

#### Протестируйте свое решение

In [6]:
class TestInstanceCountExceptionerA(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 2

    def __init__(self):
        self.a = 1

    def get(self):
        return self.a


class TestInstanceCountExceptionerB(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 3

    def __init__(self):
        self.b = 2

    def get(self):
        return self.b


class TestInstanceCountExceptionerDefaultValue(
    metaclass=InstanceCountExceptioner):
    def __init__(self):
        self.a = 3

    def get(self):
        return self.a


def test_simple():
    a = TestInstanceCountExceptionerA()
    b = TestInstanceCountExceptionerB()
    c = TestInstanceCountExceptionerDefaultValue()

    assert a.get() == 1 == a.a
    assert b.get() == 2 == b.b
    assert c.get() == 3 == c.a

    print('test_simple passed!')


def test_create():
    a2 = TestInstanceCountExceptionerA()
    b2 = TestInstanceCountExceptionerB()

    b3 = TestInstanceCountExceptionerB()

    b2.b = 20
    b3.b = 200

    assert b2.b == 20
    assert b3.b == 200

    print('test_create passed!')


def test_fail_create_a():
    try:
        TestInstanceCountExceptionerA()
        assert False
    except InstanceCountException:
        pass

    print('test_fail_create_a passed!')


def test_fail_create_b():
    try:
        TestInstanceCountExceptionerB()
        assert False
    except InstanceCountException:
        pass

    print('test_fail_create_b passed!')


def test_fail_create_default_value():
    try:
        TestInstanceCountExceptionerDefaultValue()
        assert False
    except InstanceCountException:
        pass

    print('test_fail_create_default_value passed!')


test_simple()
test_create()
test_fail_create_a()
test_fail_create_b()
test_fail_create_default_value()

test_simple passed!
test_create passed!
test_fail_create_a passed!
test_fail_create_b passed!
test_fail_create_default_value passed!


## 3. JSONClassCreator (0.6 баллов)
Напишите метакласс, который будет по json представлению строить новый класс и обратно. Класс должен уметь следующее:
* Поддерживать сохранение и получение магических функций класса.
* Поддерживать сохранение и получение обычных функций.
* Поддерживать сохранение полей со стандартными типами.
* Уберите из сохранения следующие поля и методы: ['\_\_dict\_\_', '\_\_weakref\_\_', '\_\_module\_\_', '\_\_init\_\_']
* У создаваемого класса должна быть функция to_json_str

Формат json строки должен быть следующий:

<code>
{
    "name": название класса,
    "bases": базовые классы,
    "methods": методы класса,
    "attrs": поля класса
}
</code>

Рекомендации:
* Для получения кода функций используйте модуль <a href="http://python-lab.ru/documentation/27/stdlib/inspect.html">inspect</a>.
* Для того, чтобы запустить код функций, можно использовать exec.
* Можно не исправлять ошибку типа OSError: could not get source code - возникает для функций, полученных с помощью exec.

#### Пример использования

In [2]:
import json


class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [5]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'f': 'def f(self, x):\n        print(x)', '__repr__': 'def __repr__(self):\n        return "Test(val={})".format(self.val)', '__str__': 'def __str__(self):\n        return "Test(val={})".format(self.val)'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})


In [6]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [7]:
tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

hi


(Test(val=[1, 2, 3]), None, [1, 2, 3], 'Тестовый класс')

In [10]:
tmp.__dict__

mappingproxy({'__doc__': 'Тестовый класс',
              '__module__': '__main__',
              '__repr__': <function __main__.__repr__>,
              '__str__': <function __main__.__str__>,
              'f': <function __main__.f>,
              'to_json_str': <function __main__.JSONClassCreator.to_json_str>,
              'val': [1, 2, 3]})

#### Решение

In [ ]:
import inspect


class JSONClassCreator(type):
    def __new__(mcls, json_str):
        <your code here>

    def to_json_str(cls):
        exclude = ['__dict__', '__weakref__', '__module__', '__init__']
        <your code here>

        return json.dumps({
            "name": <your code here>,
            "bases": <your code here>,
            "methods": <your code here>,
            "attrs": <your code here>
        })

    pass


#### Проверьте свое решение на примере